## **Solubility Prediction using Molecular Descriptors with RDKit**

### **Install rdkit**

In [1]:
pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 43.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install deepchem

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.2/693.2 kB 32.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os, math
import pandas as pd
import numpy as np

# for the compound descriptors
import rdkit
from rdkit import Chem
import deepchem as dc
from rdkit.Chem import Descriptors
from rdkit.Chem import inchi
from rdkit.Chem import rdMolDescriptors

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt

# Modeling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
from lightgbm import Dataset
from hyperopt import fmin, tpe, hp, Trials
import itertools

from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve, precision_score, recall_score


import warnings
warnings.filterwarnings('ignore')


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


## **Rread the Dataset**

In [4]:
df = pd.read_csv('/kaggle/input/aqueous-solubility-predictioin/train.csv')
df_test = pd.read_csv('/kaggle/input/aqueous-solubility-predictioin/test.csv')

In [5]:
df

,ID,Name,InChI,InChIKey,SMILES,Solubility,SD,Ocurrences,Group,MolWt,...,NumValenceElectrons,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,RingCount,TPSA,LabuteASA,BalabanJ,BertzCT,comp_id
0,A-2510,1-(hexyloxy)hexane,InChI=1S/C12H26O/c1-3-5-7-9-11-13-12-10-8-6-4-...,BPIUIOXAFBGMNB-UHFFFAOYSA-N,CCCCCCOCCCCCC,-4.270304,0.000000,1,G1,186.339,...,80.0,0.0,0.0,0.0,0.0,9.23,83.867160,2.758150,71.193662,1460
1,F-54,"(6-nitro-1,3-benzodioxol-5-yl)methanol",InChI=1S/C8H7NO5/c10-3-5-1-7-8(14-4-13-7)2-6(5...,XSKQKDTZQNFCCB-UHFFFAOYSA-N,C1OC2=C(O1)C=C(C(=C2)CO)[N+](=O)[O-],-3.060000,0.000000,1,G1,197.146,...,74.0,1.0,0.0,1.0,2.0,81.83,78.829756,2.540230,389.563183,8962
2,B-1415,ethyl propyl ether,"InChI=1S/C5H12O/c1-3-5-6-4-2/h3-5H2,1-2H3",NVJUHMXYKCUMQA-UHFFFAOYSA-N,CCCOCC,-0.680600,0.082783,3,G5,88.150,...,38.0,0.0,0.0,0.0,0.0,9.23,39.312565,2.339092,15.900135,4663
3,B-307,diphenylacetonitrile,InChI=1S/C14H11N/c15-11-14(12-7-3-1-4-8-12)13-...,NEBPTMCRLHKPOB-UHFFFAOYSA-N,N#CC(c1ccccc1)c2ccccc2,-2.943700,0.000000,1,G1,193.249,...,72.0,2.0,0.0,0.0,2.0,23.79,89.610961,2.392308,414.202303,3846
4,B-1164,9-methyl-9-azabicyclo[3.3.1]nonan-3-one,InChI=1S/C9H15NO/c1-10-7-3-2-4-8(10)6-9(11)5-7...,RHWSKVCZXBAWLZ-UHFFFAOYSA-N,CN1C2CCCC1CC(=O)C2,0.416700,0.000000,1,G1,153.225,...,62.0,0.0,2.0,2.0,2.0,20.31,67.568481,2.122767,162.401547,4455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6982,C-1308,n-(benzoyloxyacetyl)proline,InChI=1S/C14H15NO5/c16-12(15-8-4-7-11(15)13(17...,GZXGHZDSFNQRKK-UHFFFAOYSA-N,OC(=O)C1CCCN1C(=O)COC(=O)C2=CC=CC=C2,-1.590000,0.000000,1,G1,277.276,...,106.0,1.0,1.0,1.0,2.0,83.91,115.497288,1.892259,513.555365,7690
6983,A-1843,1-(2-ethylbutyl)cyclohexane-1-carboxylic acid,InChI=1S/C13H24O2/c1-3-11(4-2)10-13(12(14)15)8...,XRKJBJLXKLOLKS-UHFFFAOYSA-N,CCC(CC)CC1(CCCCC1)C(O)=O,-4.281695,0.000000,1,G1,212.333,...,88.0,0.0,1.0,1.0,1.0,37.30,93.068481,2.672386,200.581493,1072
6984,B-2106,kasugamycin,"InChI=1S/C14H21N3O3/c1-14(2,3)16-13(19)20-11-8...",OWNAXTAAAQTBSP-UHFFFAOYSA-N,CN(C)C(=O)Nc1cccc(OC(=O)NC(C)(C)C)c1,-2.934200,0.000000,1,G1,279.340,...,110.0,1.0,0.0,0.0,1.0,70.67,118.647088,2.687869,492.682650,5266
6985,B-2737,zolone,"InChI=1S/C12H16NO4PS2/c1-3-15-18(19,16-4-2)20-...",YAAFKGCBOCRHIF-UHFFFAOYSA-N,CCO[P](=S)(OCC)SCN1C(=O)Oc2ccccc12,-5.190500,0.000000,1,G1,333.371,...,110.0,2.0,0.0,0.0,2.0,53.60,125.715292,2.218033,671.392314,5820


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6987 entries, 0 to 6986
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   6987 non-null   object 
 1   Name                 6987 non-null   object 
 2   InChI                6987 non-null   object 
 3   InChIKey             6987 non-null   object 
 4   SMILES               6987 non-null   object 
 5   Solubility           6987 non-null   float64
 6   SD                   6987 non-null   float64
 7   Ocurrences           6987 non-null   int64  
 8   Group                6987 non-null   object 
 9   MolWt                6987 non-null   float64
 10  MolLogP              6987 non-null   float64
 11  MolMR                6987 non-null   float64
 12  HeavyAtomCount       6987 non-null   float64
 13  NumHAcceptors        6987 non-null   float64
 14  NumHDonors           6987 non-null   float64
 15  NumHeteroatoms       6987 non-null   f

In [7]:
df.describe()

,Solubility,SD,Ocurrences,MolWt,MolLogP,MolMR,HeavyAtomCount,NumHAcceptors,NumHDonors,NumHeteroatoms,...,NumValenceElectrons,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,RingCount,TPSA,LabuteASA,BalabanJ,BertzCT,comp_id
count,6987.000000,6987.000000,6987.000000,6987.000000,6987.000000,6987.000000,6987.000000,6987.000000,6987.000000,6987.000000,...,6987.000000,6987.000000,6987.000000,6987.000000,6987.000000,6987.000000,6987.000000,6987.000000,6987.000000,6987.000000
mean,-2.901533,0.067631,1.375841,268.566648,1.992814,67.338892,17.521111,3.528553,1.118935,5.237584,...,95.004437,1.079004,0.294690,0.451553,1.530557,63.202691,109.760218,2.389069,474.707556,5020.937312
std,2.377531,0.238107,1.024258,187.703886,3.569885,48.291920,12.642758,3.549758,1.457194,4.753675,...,66.658404,1.338521,0.815172,1.005877,1.634609,64.179108,79.022931,1.091260,572.552600,2889.935140
min,-13.171900,0.000000,1.000000,9.012000,-40.873200,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.581939,-0.000004,0.000000,1.000000
25%,-4.365876,0.000000,1.000000,163.182000,0.604350,40.740850,11.000000,2.000000,0.000000,3.000000,...,58.000000,0.000000,0.000000,0.000000,0.000000,26.300000,66.954471,2.000000,164.246629,2522.500000
50%,-2.622796,0.000000,1.000000,229.233000,1.951100,59.130200,15.000000,3.000000,1.000000,4.000000,...,82.000000,1.000000,0.000000,0.000000,1.000000,51.210000,93.704021,2.539539,359.261280,5023.000000
75%,-1.203250,0.000000,1.000000,322.390000,3.448950,82.267800,21.000000,4.000000,2.000000,6.000000,...,112.000000,2.000000,0.000000,1.000000,2.000000,80.640000,129.500469,3.023716,615.511782,7549.500000
max,2.137682,3.870145,38.000000,5299.456000,68.541140,1419.351700,388.000000,86.000000,21.000000,89.000000,...,2012.000000,35.000000,21.000000,21.000000,36.000000,1214.340000,2230.685124,7.517310,20720.267708,9981.000000


In [8]:
df.isna().sum()

ID                     0
Name                   0
InChI                  0
InChIKey               0
SMILES                 0
Solubility             0
SD                     0
Ocurrences             0
Group                  0
MolWt                  0
MolLogP                0
MolMR                  0
HeavyAtomCount         0
NumHAcceptors          0
NumHDonors             0
NumHeteroatoms         0
NumRotatableBonds      0
NumValenceElectrons    0
NumAromaticRings       0
NumSaturatedRings      0
NumAliphaticRings      0
RingCount              0
TPSA                   0
LabuteASA              0
BalabanJ               0
BertzCT                0
comp_id                0
dtype: int64

In [9]:
df.duplicated().sum()

0

## **Calculate molecular descriptors in rdkit**

#### **Convert list of molecules to rdkit object**

In [10]:
def get_mol_list(sol):
    mol_list= []
    for element in sol.SMILES:
      mol = Chem.MolFromSmiles(element)
      mol_list.append(mol)
    return mol_list

In [11]:
mol_list=get_mol_list(df)
mol_list_test=get_mol_list(df_test)

[14:31:33] WARNING: not removing hydrogen atom without neighbors
[14:31:33] WARNING: not removing hydrogen atom without neighbors
[14:31:33] WARNING: not removing hydrogen atom without neighbors
[14:31:33] WARNING: not removing hydrogen atom without neighbors
[14:31:33] WARNING: not removing hydrogen atom without neighbors
[14:31:33] WARNING: not removing hydrogen atom without neighbors
[14:31:33] WARNING: not removing hydrogen atom without neighbors
[14:31:33] WARNING: not removing hydrogen atom without neighbors
[14:31:33] WARNING: not removing hydrogen atom without neighbors
[14:31:33] WARNING: not removing hydrogen atom without neighbors
[14:31:33] WARNING: not removing hydrogen atom without neighbors
[14:31:33] WARNING: not removing hydrogen atom without neighbors
[14:31:33] WARNING: not removing hydrogen atom without neighbors
[14:31:33] WARNING: not removing hydrogen atom without neighbors
[14:31:33] WARNING: not removing hydrogen atom without neighbors
[14:31:33] WARNING: not r

**Calculate molecular descriptors**

To predict LogS (log of the aqueous solubility), the study by Delaney makes use of 4 molecular descriptors:

1. cLogP (Octanol-water partition coefficient)
2. MW (Molecular weight)
3. RB (Number of rotatable bonds)
4. AP (Aromatic proportion = number of aromatic atoms / total number of heavy atoms)

Unfortunately, rdkit readily computes the first 3. As for the AP descriptor, we will calculate this by manually computing the ratio of the number of aromatic atoms to the total number of heavy atoms which rdkit can compute.

In [12]:
def generate(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_MolWt = Descriptors.MolWt(mol)
        desc_NumRotatableBonds = Descriptors.NumRotatableBonds(mol)
           
        row = np.array([desc_MolLogP,
                        desc_MolWt,
                        desc_NumRotatableBonds])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MolLogP","MolWt","NumRotatableBonds"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors


##### **Aromatic proportion**

- **Number of aromatic atoms :**
    Here, we will create a custom function to calculate the Number of aromatic atoms.
    With this descriptor we can use it to subsequently calculate the AP descriptor.

In [13]:
def AromaticAtoms(m):
  aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
  aa_count = []
  for i in aromatic_atoms:
    if i==True:
      aa_count.append(1)
  sum_aa_count = sum(aa_count)
  return sum_aa_count

In [14]:
desc_AromaticAtoms = [AromaticAtoms(element) for element in mol_list]
desc_AromaticAtoms_test = [AromaticAtoms(element) for element in mol_list_test]

- **Number of heavy atoms :**
Here, we will use an existing function for calculating the Number of heavy atoms.

In [15]:
desc_HeavyAtomCount = [Descriptors.HeavyAtomCount(element) for element in mol_list]
desc_HeavyAtomCount_test = [Descriptors.HeavyAtomCount(element) for element in mol_list_test]

**Computing the Aromatic Proportion (AP) descriptor**

In [16]:
desc_AromaticProportion = [AromaticAtoms(element)/Descriptors.HeavyAtomCount(element) for element in mol_list]
desc_AromaticProportion_test = [AromaticAtoms(element)/Descriptors.HeavyAtomCount(element) for element in mol_list_test]

### **Creating DataFrame**

In [17]:
discriptors =generate(df.SMILES)
discriptors_test=generate(df_test.SMILES)

[14:31:35] WARNING: not removing hydrogen atom without neighbors
[14:31:35] WARNING: not removing hydrogen atom without neighbors
[14:31:35] WARNING: not removing hydrogen atom without neighbors
[14:31:35] WARNING: not removing hydrogen atom without neighbors
[14:31:35] WARNING: not removing hydrogen atom without neighbors
[14:31:35] WARNING: not removing hydrogen atom without neighbors
[14:31:35] WARNING: not removing hydrogen atom without neighbors
[14:31:35] WARNING: not removing hydrogen atom without neighbors
[14:31:35] WARNING: not removing hydrogen atom without neighbors
[14:31:35] WARNING: not removing hydrogen atom without neighbors
[14:31:35] WARNING: not removing hydrogen atom without neighbors
[14:31:35] WARNING: not removing hydrogen atom without neighbors
[14:31:35] WARNING: not removing hydrogen atom without neighbors
[14:31:35] WARNING: not removing hydrogen atom without neighbors
[14:31:35] WARNING: not removing hydrogen atom without neighbors
[14:31:35] WARNING: not r

**RDKit molecular descriptors based features**
* Use DeepChem featurizer with RDKitDescriptors
* Generates 208 features with both use_fragment and ipc_avg
* Here, we generate 123 different descriptors in total for evey molecules

**For SMILES**

In [18]:
# # DeepChem RDKit descriptors
# rdkit_featurizer = dc.feat.RDKitDescriptors(use_fragment=False, ipc_avg=False)
# features = rdkit_featurizer(df.SMILES) # with one molecule
# features_test = rdkit_featurizer(df_test.SMILES) # with one molecule

In [19]:
# features.shape

In [20]:
# # Creating the feature dataset

# column_names = rdkit_featurizer.descriptors

# df0 = pd.DataFrame(data=features)
# df0.columns = column_names

# # adding molecule ids and solubility columns
# # df0.insert(0, "ID", df.ID)
# df0["Solubility"] = df.Solubility

In [21]:
# # Creating the feature dataset

# column_names_test = rdkit_featurizer.descriptors

# df1 = pd.DataFrame(data=features_test)
# df1.columns = column_names

# # adding molecule ids and solubility columns
# # df1.insert(0, "ID", df_test.ID)

In [22]:
# df0.isna().sum().sum()

In [23]:
# df0.head()

**For InCHI**

In [24]:
# # Define a function to convert an InChI string to an RDKit molecule object
# def inchi_to_mol(inchi_str):
#     return Chem.inchi.MolFromInchi(inchi_str)

# # Apply the inchi_to_mol() function to the 'InChI' column in your DataFrame
# df['Mol'] = df['InChI'].apply(inchi_to_mol)
# df_test['Mol'] = df_test['InChI'].apply(inchi_to_mol)

# # Define a function to generate InChI-based vectors from RDKit molecules
# def generate_inchi_vector(mol, max_length=64):
#     inchi_str = inchi.MolToInchi(mol)
#     inchi_dict = rdMolDescriptors.GetHashedAtomPairFingerprint(mol, nBits=max_length, includeChirality=True)
#     inchi_dict_nonzero = inchi_dict.GetNonzeroElements()
#     inchi_vector = np.zeros(max_length)
#     inchi_vector[list(inchi_dict_nonzero.keys())] = list(inchi_dict_nonzero.values())
#     return inchi_vector

# # Apply the generate_inchi_vector() function to the 'Mol' column in your DataFrame
# df['InChI_Vector'] = df['Mol'].apply(generate_inchi_vector)
# df_test['InChI_Vector'] = df_test['Mol'].apply(generate_inchi_vector)

# # Convert the 'InChI_Vector' column to a numpy array for downstream analysis
# InChI_features = np.array(df['InChI_Vector'].to_list())
# InChI_features_test = np.array(df_test['InChI_Vector'].to_list())

# df['InChI_Vector'] = InChI_features
# df_test['InChI_Vector'] = InChI_features_test

**gathering all good features**

In [25]:
df['AromaticProportion']=desc_AromaticProportion
df_test['AromaticProportion']=desc_AromaticProportion_test

In [26]:
X= pd.concat([df['AromaticProportion'],discriptors], axis=1)
X_test= pd.concat([df_test['AromaticProportion'],discriptors_test], axis=1)

In [27]:
X.columns

Index(['AromaticProportion', 'MolLogP', 'MolWt', 'NumRotatableBonds'], dtype='object')

In [28]:
# # check if 'MolLogP' column exists in the dataframe
# if 'MolLogP' in df0.columns:
#     print("Column 'MolLogP' exists in the DataFrame")
# else:
#     print("Column 'MolLogP' does not exist in the DataFrame")

# # check if 'MolWt' column exists in the dataframe
# if 'MolWt' in df0.columns:
#     print("Column 'MolWt' exists in the DataFrame")
# else:
#     print("Column 'MolWt' does not exist in the DataFrame")

# # check if 'NumRotatableBonds' column exists in the dataframe
# if 'NumRotatableBonds' in df0.columns:
#     print("Column 'NumRotatableBonds' exists in the DataFrame")
# else:
#     print("Column 'NumRotatableBonds' does not exist in the DataFrame")

In [29]:
# df0=df0.drop(['MolLogP','MolWt','NumRotatableBonds'],axis=1)
# df1=df1.drop(['MolLogP','MolWt','NumRotatableBonds'],axis=1)

In [30]:
# # X= pd.concat([X,df0], axis=1)
# # X_test= pd.concat([X_test,df1], axis=1)
# X= pd.concat([df['comp_id'],df0], axis=1)
# X_test= pd.concat([df_test['comp_id'],df1], axis=1)

In [31]:
X= pd.concat([df['comp_id'],X], axis=1)
X_test= pd.concat([df_test['comp_id'],X_test], axis=1)

In [32]:
# X= pd.concat([df['AromaticProportion'],X], axis=1)
# X_test= pd.concat([df_test['AromaticProportion'],X_test], axis=1)

In [33]:
X.head()

,comp_id,AromaticProportion,MolLogP,MolWt,NumRotatableBonds
0,1460,0.000000,4.16360,186.339,10.0
1,8962,0.428571,0.81580,197.146,2.0
2,4663,0.000000,1.43290,88.150,3.0
3,3846,0.800000,3.34208,193.249,2.0
4,4455,0.000000,1.20220,153.225,0.0


In [34]:
X.isna().sum().sum()

0

In [35]:
missing_columns = set(X.columns) - set(X_test.columns)
print(missing_columns)

set()


In [36]:
# X = X.drop(columns=["Solubility"])

y=df.Solubility

In [37]:
# drop columns with null values in X
X = X.dropna(axis=1)

# drop the same columns in X_test
X_test = X_test[X.columns]

In [38]:
X.isna().sum().sum()

0

In [39]:
np.isinf(X).sum().sum()

0

In [40]:
X.shape

(6987, 5)

In [41]:
X_test.shape

(2995, 5)

In [42]:
comp_id=df_test.comp_id

In [43]:
# # Calculate the percentage of zeros in each column
# zero_percentages = (X == 0).sum() / len(df)

# # Iterate through each column to check if the zero percentage is more than 50%
# for column in zero_percentages.index:
#     if zero_percentages[column] > 0.5:
#         # If the zero percentage is more than 50%, drop the column
#         X = X.drop(column, axis=1)
#         X_test=X_test.drop(column, axis=1)
#         print("Dropped column:", column)


In [44]:
# X_test.shape

In [45]:
# X.shape

In [46]:
# for col in X.columns :
#     print(col)

### **The Quality-Oriented Data Selection**

In [47]:
# # 1- Split the dataset into training and validation sets.
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# train_data = Dataset(X_train, label=y_train)
# val_data = Dataset(X_val, label=y_val)

In [48]:
# # 2- Train a machine learning model on the training set using all available data.
# params = {
#             "objective": "regression",
#             "is_unbalance": "true",
#             "boosting_type": "dart",
#             "bagging_ratio": 0.6,
#             "feature_fraction": 0.6,
#             "metric": ["rmse"],
# }
# # training a LightGBM model
# model = lgb.train(params=params,
#                  train_set=train_data,
#                  valid_sets=[val_data, train_data],
#                   num_boost_round=1500,
#                   verbose_eval=100
#                  )


In [49]:
# # 3- Evaluate the performance of the model on the validation set.
# y_pred=model.predict(X_val)
# print("\nThe RMSE :",mean_squared_error(y_val, y_pred,squared=False))

In [50]:
# # 4- Calculate a quality score for each data point based on its prediction error and uncertainty.
# errors = y_val - y_pred
# std = np.std(errors)
# scores = np.abs(errors) / std

In [51]:
# # 5- Select a subset of high-quality data points based on their quality scores.
# num_points = int(len(X) * 0.5)  # increase the number of high-quality data points to 50%
# indices = np.argsort(scores)[-num_points:]
# X_high_quality = X.reset_index(drop=True).iloc[indices]
# y_high_quality = y.reset_index(drop=True).iloc[indices]

In [52]:
# X_high_quality.shape

In [53]:
# # 6- Train a new machine learning model using only the selected high-quality data points.
# X_train, X_val, y_train, y_val = train_test_split(X_high_quality, y_high_quality, test_size=0.2, random_state=42)

# # Define the hyperparameter search space
# space = {
#     'num_leaves': hp.quniform('num_leaves', 10, 100, 1),
#     'learning_rate': hp.loguniform('learning_rate', -5, 0),
#     'min_child_samples': hp.quniform('min_child_samples', 5, 50, 1),
#     'subsample': hp.uniform('subsample', 0.1, 1),
#     'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1),
#     'reg_alpha': hp.loguniform('reg_alpha', -10, 1),
#     'reg_lambda': hp.loguniform('reg_lambda', -10, 1)
# }

# # Define the objective function to minimize (in this case, the root mean squared error)
# def objective(params):
#     params = {
#         'num_leaves': int(params['num_leaves']),
#         'learning_rate': params['learning_rate'],
#         'min_child_samples': int(params['min_child_samples']),
#         'subsample': params['subsample'],
#         'metric': 'rmse',
#         'colsample_bytree': params['colsample_bytree'],
#         'reg_alpha': params['reg_alpha'],
#         'reg_lambda': params['reg_lambda']
#     }
    
#     model = lgb.LGBMRegressor(**params, n_estimators=1000, n_jobs=-1)
#     model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=False)
#     y_pred = model.predict(X_val)
#     rmse = mean_squared_error(y_val, y_pred, squared=False)
#     return rmse

# # Run the hyperparameter search using Tree-structured Parzen Estimator (TPE)
# trials = Trials()
# best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials, verbose=1)

# # Print the hyperparameters with the best loss
# print("Best hyperparameters:", best)

# # Train the final model using the best hyperparameters
# best_params = {
#     'num_leaves': int(best['num_leaves']),
#     'learning_rate': best['learning_rate'],
#     'min_child_samples': int(best['min_child_samples']),
#     'subsample': best['subsample'],
#     'colsample_bytree': best['colsample_bytree'],
#     'reg_alpha': best['reg_alpha'],
#     'reg_lambda': best['reg_lambda']
# }

# model = lgb.LGBMRegressor(**best_params, n_estimators=1000, n_jobs=-1)
# model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=150, verbose=False)

# # Evaluate the final model on the validation set
# y_pred = model.predict(X_val)
# rmse = mean_squared_error(y_val, y_pred, squared=False)
# print("Final root mean squared error:", rmse)


#### **Scaling the Data**

In [54]:
# # first, make the log transformation
# X_log = np.log(X + 1e-10)
# X_test_log = np.log(X_test + 1e-10)

# # Apply RobustScaler to all the data
# scaler = RobustScaler()
# X_scaled = scaler.fit_transform(X)
# X_test_scaled = scaler.transform(X_test)

# # Apply StandardScaler to all the data
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
# X_test_scaled = scaler.transform(X_test)

# # Apply MinMaxScaler to all the data
# scaler = MinMaxScaler()
# X_scaled = scaler.fit_transform(X)
# X_test_scaled = scaler.transform(X_test)

In [55]:
# Stop Here to Train the models...........

## **Modeling**

#### **PCA**

In [56]:
# # Create a StandardScaler object and fit_transform the data
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
# X_test_scaled = scaler.transform(X_test)

# # Create a PCA object with 2 components and fit_transform the scaled data
# pca = PCA(n_components=50)
# X_pca = pca.fit_transform(X_scaled)
# X_test_pca = pca.transform(X_test_scaled)

In [57]:
# X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.1, random_state=42)

In [58]:
# X_pca.shape

### **Data split**

In [59]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.01, random_state=42)

#### **Linear Regression Model**

In [60]:
# # Create a linear regression model and fit it to the scaled training data
# regressor = LinearRegression()
# regressor.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred = regressor.predict(X_val)

# # Evaluate the model's performance
# score = mean_squared_error(y_val, y_pred,squared=False)
# print(f'The RMSE : {score}')

#### **LightGBM model**

In [61]:
# Define the hyperparameter search space
space = {
    'num_leaves': hp.quniform('num_leaves', 10, 100, 1),
    'learning_rate': hp.loguniform('learning_rate', -5, 0),
    'min_child_samples': hp.quniform('min_child_samples', 5, 50, 1),
    'subsample': hp.uniform('subsample', 0.1, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1),
    'reg_alpha': hp.loguniform('reg_alpha', -10, 1),
    'reg_lambda': hp.loguniform('reg_lambda', -10, 1)
}

# Define the objective function to minimize (in this case, the root mean squared error)
def objective(params):
    params = {
        'num_leaves': int(params['num_leaves']),
        'learning_rate': params['learning_rate'],
        'min_child_samples': int(params['min_child_samples']),
        'subsample': params['subsample'],
        'metric': 'rmse',
        'colsample_bytree': params['colsample_bytree'],
        'reg_alpha': params['reg_alpha'],
        'reg_lambda': params['reg_lambda']
    }
    
    model = lgb.LGBMRegressor(**params, n_estimators=1000, n_jobs=-1)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=False)
    y_pred = model.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    return rmse

# Run the hyperparameter search using Tree-structured Parzen Estimator (TPE)
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials, verbose=1)

# Print the hyperparameters with the best loss
print("Best hyperparameters:", best)

# Train the final model using the best hyperparameters
best_params = {
    'num_leaves': int(best['num_leaves']),
    'learning_rate': best['learning_rate'],
    'min_child_samples': int(best['min_child_samples']),
    'subsample': best['subsample'],
    'colsample_bytree': best['colsample_bytree'],
    'reg_alpha': best['reg_alpha'],
    'reg_lambda': best['reg_lambda']
}

model = lgb.LGBMRegressor(**best_params, n_estimators=1000, n_jobs=-1)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=150, verbose=False)

# Evaluate the final model on the validation set
y_pred = model.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)
print("Final root mean squared error:", rmse)

100%|██████████| 100/100 [01:13<00:00,  1.37trial/s, best loss: 0.8756169500031734]
Best hyperparameters: {'colsample_bytree': 0.9962150743079714, 'learning_rate': 0.0911831272211416, 'min_child_samples': 23.0, 'num_leaves': 93.0, 'reg_alpha': 0.0064780076942777655, 'reg_lambda': 0.11347419395068079, 'subsample': 0.14848207653717005}
Final root mean squared error: 0.8756169500031734


#### **CatBoost**

In [62]:
# from hyperopt import fmin, tpe, hp, Trials
# from catboost import CatBoostRegressor

# # Define the hyperparameter search space
# space = {
#     'iterations': hp.quniform('iterations', 100, 1000, 1),
#     'learning_rate': hp.loguniform('learning_rate', -5, 0),
#     'depth': hp.quniform('depth', 1, 10, 1),
#     'l2_leaf_reg': hp.loguniform('l2_leaf_reg', -5, 2),
#     'subsample': hp.uniform('subsample', 0.1, 1),
#     'colsample_bylevel': hp.uniform('colsample_bylevel', 0.1, 1),
#     'bagging_temperature': hp.loguniform('bagging_temperature', -5, 2)
# }

# # Define the objective function to minimize (in this case, the mean squared error)
# def objective(params):
#     model = CatBoostRegressor(
#         iterations=int(params['iterations']),
#         learning_rate=params['learning_rate'],
#         depth=int(params['depth']),
#         l2_leaf_reg=params['l2_leaf_reg'],
#         subsample=params['subsample'],
#         colsample_bylevel=params['colsample_bylevel'],
#         bagging_temperature=params['bagging_temperature'],
#         random_state=42,
#         eval_metric='RMSE',
#         verbose=False
#     )
    
#     model.fit(
#         X_train, y_train,
#         eval_set=(X_val, y_val),
#         early_stopping_rounds=100,
#         verbose=False
#     )
    
#     y_pred = model.predict(X_val)
#     mse = mean_squared_error(y_val, y_pred)
#     return mse

# # Run the hyperparameter search using Tree-structured Parzen Estimator (TPE)
# trials = Trials()
# best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials, verbose=1)

# # Print the hyperparameters with the best loss
# print("Best hyperparameters:", best)

# # Train the final model using the best hyperparameters
# best_params = {
#     'iterations': int(best['iterations']),
#     'learning_rate': best['learning_rate'],
#     'depth': int(best['depth']),
#     'l2_leaf_reg': best['l2_leaf_reg'],
#     'subsample': best['subsample'],
#     'colsample_bylevel': best['colsample_bylevel'],
#     'bagging_temperature': best['bagging_temperature'],
#     'random_state': 42,
#     'eval_metric': 'RMSE',
#     'verbose': False
# }

# model = CatBoostRegressor(**best_params)
# model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=100, verbose=False)

# # Evaluate the final model on the validation set
# y_pred = model.predict(X_val)
# rmse = mean_squared_error(y_val, y_pred, squared=False)
# print("Final root mean squared error:", rmse)


#### **XGBoost Model**

In [63]:
# import xgboost as xgb
# from hyperopt import fmin, tpe, hp, Trials
# from sklearn.metrics import mean_squared_error

# #Define the hyperparameter search space
# space = {
# 'max_depth': hp.choice('max_depth', range(1, 10)),
# 'learning_rate': hp.loguniform('learning_rate', -5, 0),
# 'subsample': hp.uniform('subsample', 0.1, 1),
# 'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1),
# 'gamma': hp.loguniform('gamma', -10, 1),
# 'reg_alpha': hp.loguniform('reg_alpha', -10, 1),
# 'reg_lambda': hp.loguniform('reg_lambda', -10, 1),
# 'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
# 'objective': 'reg:squarederror'
# }

# #Define the objective function to minimize (in this case, the mean squared error)
# def objective(params):
#     params['max_depth'] = int(params['max_depth'])
#     dtrain = xgb.DMatrix(X_train, label=y_train)
#     dval = xgb.DMatrix(X_val, label=y_val)
#     evallist = [(dval, 'eval')]
#     num_round = 1000

#     model = xgb.train(params, dtrain, num_round, evallist, early_stopping_rounds=100, verbose_eval=False)
#     y_pred = model.predict(dval)
#     mse = mean_squared_error(y_val, y_pred)
#     return mse

# #Run the hyperparameter search using Tree-structured Parzen Estimator (TPE)
# trials = Trials()
# best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials, verbose=1)

# #Print the hyperparameters with the best loss
# print("Best hyperparameters:", best)

# #Train the final model using the best hyperparameters
# best_params = {
# 'max_depth': int(best['max_depth']),
# 'learning_rate': best['learning_rate'],
# 'subsample': best['subsample'],
# 'colsample_bytree': best['colsample_bytree'],
# 'gamma': best['gamma'],
# 'reg_alpha': best['reg_alpha'],
# 'reg_lambda': best['reg_lambda'],
# 'min_child_weight': int(best['min_child_weight']),
# 'objective': 'reg:squarederror'
# }

# dtrain = xgb.DMatrix(X_train, label=y_train)
# dval = xgb.DMatrix(X_val, label=y_val)

# num_round = 1000
# model = xgb.train(best_params, dtrain, num_round, evals=[(dval, 'eval')], early_stopping_rounds=100, verbose_eval=False)

# #Evaluate the final model on the test set
# y_pred = model.predict(dval)
# rmse = mean_squared_error(y_val, y_pred, squared=False)
# print("Final root mean squared error:", rmse)

### **Consensus Models:**
* Neural Nets
* XGBoost
* Random Forest

In [64]:
# from sklearn.model_selection import cross_val_predict, KFold
# from sklearn.neural_network import MLPRegressor
# from xgboost import XGBRegressor
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error

# # Define the models
# model_nn = MLPRegressor(activation='tanh', hidden_layer_sizes=(500,), max_iter=1000, alpha=0.001, random_state=42, solver='adam')
# model_xgb = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
# model_rf = RandomForestRegressor(n_estimators=100, random_state=42)

# # Fit the models using cross-fold validation
# cv = KFold(n_splits=5, shuffle=True, random_state=42)

# y_pred_nn = cross_val_predict(model_nn, X_train, y_train, cv=cv)
# y_pred_xgb = cross_val_predict(model_xgb, X_train, y_train, cv=cv)
# y_pred_rf = cross_val_predict(model_rf, X_train, y_train, cv=cv)

# # Calculate the consensus results
# y_pred_consensus = (y_pred_nn + y_pred_xgb + y_pred_rf) / 3

# # Evaluate the performance of each individual model and the consensus model
# print("Neural Net RMSE:", mean_squared_error(y_train, y_pred_nn, squared=False))
# print("XGBoost RMSE:", mean_squared_error(y_train, y_pred_xgb, squared=False))
# print("Random Forest RMSE:", mean_squared_error(y_train, y_pred_rf, squared=False))
# print("Consensus RMSE:", mean_squared_error(y_train, y_pred_consensus, squared=False))

In [65]:
# def grid_search_mpnn(train_set, hyper_params, folds=5):

#     params = list(map(lambda key: hyper_params[key], hyper_params.keys()))
#     n_of_tries = len(list(itertools.product(*params)))
#     search_results = []

#     # split dataset into folds
#     splitter = dc.splits.RandomSplitter()
#     fold_sets = splitter.k_fold_split(train_set, folds)

#     # save best hyperparams
#     best_score = 1e10
#     best_params = None
    
#     # try all possible combinations of hyperparams
#     for i, (batch_size, n_atom_feat, n_pair_feat, n_hidden) in enumerate(itertools.product(*params)):

#         rmse_scores = []

#         for train, valid in fold_sets:
#             transformers = [dc.trans.NormalizationTransformer(transform_y=True, dataset=train, move_mean=True)]

#             # preprocess data
#             for transformer in transformers:
#                 train = transformer.transform(train)
#                 valid = transformer.transform(valid)
                
#             print(f"train.shape: {train.X.shape}")
#             print(f"valid.shape: {valid.X.shape}")

#             # intantiate and fit model
#             model = dc.models.MPNNModel(1, mode='regression', batch_size=batch_size, use_queue=False, n_atom_feat=n_atom_feat, n_pair_feat=n_pair_feat, n_hidden=n_hidden, learning_rate=0.0001, T=3, M=5)
#             model.fit(train, nb_epoch=50, checkpoint_interval=100)
            
#             # evaluate model
#             metric = dc.metrics.Metric(dc.metrics.rms_score, np.mean)
#             rmse = model.evaluate(valid, [metric], transformers)['mean-rms_score']
#             rmse_scores.append(rmse)
        
#         average_rmse = np.mean(rmse_scores)

#         # save best hyperparams
#         if average_rmse < best_score:
#             best_score = average_rmse
#             best_params = (batch_size, n_atom_feat, n_pair_feat, n_hidden)

#         search_results.append([average_rmse, batch_size, n_atom_feat, n_pair_feat, n_hidden])
#         print_progress(i, n_of_tries)

#     search_results = pd.DataFrame(search_results, columns=['rmse', 'batch_size', 'n_atom_feat', 'n_pair_feat', 'n_hidden']).sort_values(by='rmse')
#     return search_results, best_params


In [66]:
# # load esol dataset from csv
# tasks = ['Solubility']
# loader = dc.data.CSVLoader(tasks=tasks, feature_field='SMILES', featurizer=dc.feat.WeaveFeaturizer())
# dataset = loader.create_dataset('/kaggle/input/aqueous-solubility-predictioin/train.csv')
# loader = dc.data.CSVLoader(tasks=[], feature_field="SMILES", featurizer=dc.feat.WeaveFeaturizer())
# test_set=loader.create_dataset('/kaggle/input/aqueous-solubility-predictioin/test.csv')
# # split esol dataset
# splitter = dc.splits.RandomSplitter()
# train_set, val_set = splitter.train_test_split(dataset, frac_train=0.8, seed=0)

In [67]:
# hyper_params = {
#     'batch_size': [32, 16],
#     'n_atom_feat': [75],
#     'n_pair_feat': [14],
#     'n_hidden': [100]
# }

# search_results, (batch_size, n_atom_feat, n_pair_feat, n_hidden) = grid_search_mpnn(train_set, hyper_params)

In [68]:
# print(search_results)

In [69]:
# batch_size, n_atom_feat, n_pair_feat, n_hidden=16,75,14,100

In [70]:
# transformers = [dc.trans.NormalizationTransformer(transform_y=True, dataset=train_set, move_mean=True)]

# # preprocess data
# for transformer in transformers:
#     train_set = transformer.transform(train_set)
#     val_set = transformer.transform(val_set)
#    # test_set = transformer.transform(test_set)

# # intantiate and fit model
# model = dc.models.MPNNModel(1, mode='regression', batch_size=batch_size, use_queue=False, n_atom_feat=n_atom_feat, n_pair_feat=n_pair_feat, n_hidden=n_hidden, learning_rate=0.0001, T=3, M=5)
# model.fit(train_set, nb_epoch=50, checkpoint_interval=100)

# # evaluate model
# metric = [
#     dc.metrics.Metric(dc.metrics.rms_score, np.mean),
#     dc.metrics.Metric(dc.metrics.mae_score, np.mean),
#     dc.metrics.Metric(dc.metrics.pearson_r2_score, np.mean)
# ]
# train_scores = model.evaluate(train_set, metric, transformers)
# val_scores = model.evaluate(val_set, metric, transformers)

# print("Train scores")
# print(train_scores)

# print("Val scores")
# print(val_scores)

# results['mpnn'] = test_scores

### **Making Prediction for Submission**

In [71]:
# Stop Here to choose the best model and ReTrain on the whole Data...........

In [72]:
#final_train_data = Dataset(X_scaled, label=y)
params = {
            "objective": "regression",
            "is_unbalance": "true",
            "boosting_type": "dart",
            "bagging_ratio": 0.6,
            "feature_fraction": 0.6,
            "metric": ["rmse"],
}

# training a LightGBM model

# model = lgb.train(params=best_params,
#                  train_set=Dataset(X, label=y),
#                  num_boost_round=1500,
#                  verbose_eval=100
#                  )

# Evaluation
y_pred_lgb=model.predict(X)

print("\nThe RMSE :",mean_squared_error(y, y_pred_lgb,squared=False))


The RMSE : 0.33444829512524454


In [73]:
y_test_predicted = model.predict(X_test)

df_test['Solubility'] = y_test_predicted
df_test['comp_id']=comp_id

In [74]:
df_test[['comp_id', 'Solubility']].to_csv('/kaggle/working/submission.csv', index=False)

In [75]:
df_test[['comp_id', 'Solubility']]

,comp_id,Solubility
0,841,-4.484597
1,6775,-4.547861
2,5670,-0.271033
3,3274,0.442704
4,7406,-3.640952
...,...,...
2990,1693,-8.420485
2991,8867,-5.040647
2992,9035,-2.170398
2993,7608,-2.322797
